In [52]:
#!/usr/bin/env python
"""
Your task is to process the supplied file and use the csv module to extract data from it.
The data comes from NREL (National Renewable Energy Laboratory) website. Each file
contains information from one meteorological station, in particular - about amount of
solar and wind energy for each hour of day.

Note that the first line of the datafile is neither data entry, nor header. It is a line
describing the data source. You should extract the name of the station from it.

The data should be returned as a list of lists (not dictionaries).
You can use the csv modules "reader" method to get data in such format.
Another useful method is next() - to get the next line from the iterator.
You should only change the parse_file function.
"""

import csv
import os

DATADIR = ""
DATAFILE = "745090.csv"

def parse_file(datafile):
    name = ""
    data = []
    with open(datafile,'rb') as f:
        i=0
        for line in f:
            line = line.decode("ASCII")
            if i==0:
                first = [str(x) for x in line.split('\"')]
                name = first[1]
                i+=1
                continue
            if i==1:
                i+=1
                continue
            y = [str(x) for x in line.split(",")]
            data.append(y)
            i+=1
        
        name = first[1]
    return (name, data)

def test():
    datafile = os.path.join(DATADIR, DATAFILE)
    name, data = parse_file(datafile)
    print(name)
    
    assert name == "MOUNTAIN VIEW MOFFETT FLD NAS"
    assert data[0][1] == "01:00"
    assert data[2][0] == "01/01/2005"
    assert data[2][5] == "2"

if __name__ == "__main__":
    test()

MOUNTAIN VIEW MOFFETT FLD NAS


In [47]:
# -*- coding: utf-8 -*-
# Find the time and value of max load for each of the regions
# COAST, EAST, FAR_WEST, NORTH, NORTH_C, SOUTHERN, SOUTH_C, WEST
# and write the result out in a csv file, using pipe character | as the delimiter.
# An example output can be seen in the "example.csv" file.

import xlrd
import os
import csv
from zipfile import ZipFile

datafile = "2013_ERCOT_Hourly_Load_Data.xls"
outfile = "2013_Max_Loads.csv"


def open_zip(datafile):
    with ZipFile('{0}.zip'.format(datafile), 'r') as myzip:
        myzip.extractall()


def parse_file(datafile):
    workbook = xlrd.open_workbook(datafile)
    sheet = workbook.sheet_by_index(0)
    data = []
    # YOUR CODE HERE
    # Remember that you can use xlrd.xldate_as_tuple(sometime, 0) to convert
    # Excel date to Python tuple of (year, month, day, hour, minute, second)
    
    data.append(["Station", "Year", "Month", "Day", "Hour", "Max Load"])
    for c in range(1, (sheet.ncols - 1)):
        maxval = 0
        row_num = 0
        i = 1
        region = str(sheet.cell_value(0, c)).strip('u')
        for r in range(1, sheet.nrows):
            if sheet.cell_value(r, c) >= maxval:
                maxval = sheet.cell_value(r, c)
                row_num = i
            i += 1
        date = xlrd.xldate_as_tuple(sheet.cell_value(row_num, 0), 0)
        year = date[0]
        month = date[1]
        day = date[2]
        hour = date[3]
        data.append([region, year, month, day, hour, maxval])
    print (data)
    return data

def save_file(data, filename):
    with open(filename, 'wb') as df:
        writer = csv.writer(df, delimiter = "|")

        for row in range(len(data)):
            print(data[row])
            x = [val.encode('ASCII') for val in data[row]]
            writer.writerow(x)

    
def test():
#     open_zip(datafile)
    data = parse_file(datafile)
    save_file(data, outfile)

    number_of_rows = 0
    stations = []

    ans = {'FAR_WEST': {'Max Load': '2281.2722140000024',
                        'Year': '2013',
                        'Month': '6',
                        'Day': '26',
                        'Hour': '17'}}
    correct_stations = ['COAST', 'EAST', 'FAR_WEST', 'NORTH',
                        'NORTH_C', 'SOUTHERN', 'SOUTH_C', 'WEST']
    fields = ['Year', 'Month', 'Day', 'Hour', 'Max Load']

    with open(outfile) as of:
        csvfile = csv.DictReader(of, delimiter="|")
        for line in csvfile:
            station = line['Station']
            if station == 'FAR_WEST':
                for field in fields:
                    # Check if 'Max Load' is within .1 of answer
                    if field == 'Max Load':
                        max_answer = round(float(ans[station][field]), 1)
                        max_line = round(float(line[field]), 1)
                        assert max_answer == max_line

                    # Otherwise check for equality
                    else:
                        assert ans[station][field] == line[field]

            number_of_rows += 1
            stations.append(station)

        # Output should be 8 lines not including header
        assert number_of_rows == 8

        # Check Station Names
        assert set(stations) == set(correct_stations)

        
if __name__ == "__main__":
    test()

[['Station', 'Year', 'Month', 'Day', 'Hour', 'Max Load'], ['COAST', 2013, 8, 13, 17, 18779.025510000003], ['EAST', 2013, 8, 5, 17, 2380.1654089999956], ['FAR_WEST', 2013, 6, 26, 17, 2281.2722140000024], ['NORTH', 2013, 8, 7, 17, 1544.7707140000005], ['NORTH_C', 2013, 8, 7, 18, 24415.570226999993], ['SOUTHERN', 2013, 8, 8, 16, 5494.157645], ['SOUTH_C', 2013, 8, 8, 18, 11433.30491600001], ['WEST', 2013, 8, 7, 17, 1862.6137649999998]]
['Station', 'Year', 'Month', 'Day', 'Hour', 'Max Load']


TypeError: a bytes-like object is required, not 'str'